In [68]:
#必要なモジュールの読み込み

import pandas as pd
import numpy as np
from scipy import stats

In [69]:
#じゃんけんデータが格納されたcsvファイルを読み込み

df = pd.read_csv('sazae_data.csv',header=None)

df.columns = ['今回']

df.head()

,今回
0,チョキ
1,グー
2,グー
3,パー
4,チョキ


In [70]:
#m回前までの手を作成(今回は５)
m = 5
df1 = df.copy()

for i in range(m):
        df1['{}回前'.format(i+1)] = df1['今回'].shift(i+1)


df1.head()

,今回,1回前,2回前,3回前,4回前,5回前
0,チョキ,None,None,None,None,None
1,グー,チョキ,None,None,None,None
2,グー,グー,チョキ,None,None,None
3,パー,グー,グー,チョキ,None,None
4,チョキ,パー,グー,グー,チョキ,None


In [3]:
#連関係数を求める関数を作成

def cramersV(x, y):
    table = np.array(pd.crosstab(x, y)).astype(np.float32)
    n = table.sum()
    colsum = table.sum(axis=0)
    rowsum = table.sum(axis=1)
    expect = np.outer(rowsum, colsum) / n
    chisq = np.sum((table - expect) ** 2 / expect)
    return np.sqrt(chisq / (n * (np.min(table.shape) - 1)))

In [71]:
#連関係数を求め、一番大きいものを確認（1回前との関係が最も大きいとわかる）
li = []

for i in range(m):
    V = cramersV(df1['今回'], df1['{}回前'.format(i+1)])
    li.append(V)
    
print(li)
max_cross = li.index(max(li))
print(max_cross)


[0.20459544669717866, 0.19025693089217124, 0.11642798824891522, 0.07631801563836249, 0.04290543374679991]
0


In [72]:
#連関係数が最も大きいもののクロス集計表
df2 = pd.crosstab(df1['今回'],df1['{}回前'.format(max_cross +1)])
df2

1回前,グー,チョキ,パー
今回,,,
グー,109,224,192
チョキ,227,110,235
パー,189,239,108


In [15]:
# 状態に対する次の手の確率を計算する関数
def next_hand_probabilities(df, prev_hand):
    prev_total = df[prev_hand].sum()  
    gu_prob = df['グー'].loc[prev_hand] / prev_total
    pa_prob = df['パー'].loc[prev_hand] / prev_total
    cho_prob = df['チョキ'].loc[prev_hand] / prev_total
    # 確率を正規化する
    total_prob = gu_prob + pa_prob + cho_prob
    gu_prob /= total_prob
    pa_prob /= total_prob
    cho_prob /= total_prob
    return gu_prob, pa_prob, cho_prob

# 次の手をランダムに選択する関数
def choose_next_hand(df, prev_hand):
    gu_prob, pa_prob, cho_prob = next_hand_probabilities(df, prev_hand)
    prob_list = [gu_prob, pa_prob, cho_prob]
    next_hand = np.random.choice(['グー', 'パー', 'チョキ'], p=prob_list)
    return next_hand

In [79]:
#次の放送で出す手を予測。これをpredcount回繰り返し、最頻値を出力。

prev_hand = df.iloc[-1]['今回']
next_list = []
predcount = 100


for i in range(predcount):
    next_hand = choose_next_hand(df2, prev_hand)
    next_list.append(next_hand)

stats.mode(next_list)

/var/folders/s_/nhcgq_zn3x7blfst6_jz0mqw0000gn/T/ipykernel_25484/4108723278.py:12: FutureWarning: Unlike other reduction functions (e.g. `skew`, `kurtosis`), the default behavior of `mode` typically preserves the axis it acts along. In SciPy 1.11.0, this behavior will change: the default value of `keepdims` will become False, the `axis` over which the statistic is taken will be eliminated, and the value None will no longer be accepted. Set `keepdims` to True or False to avoid this warning.
  stats.mode(next_list)
/var/folders/s_/nhcgq_zn3x7blfst6_jz0mqw0000gn/T/ipykernel_25484/4108723278.py:12: DeprecationWarning: Support for non-numeric arrays has been deprecated as of SciPy 1.9.0 and will be removed in 1.11.0. `pandas.DataFrame.mode` can be used instead, see https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.mode.html.
  stats.mode(next_list)


ModeResult(mode=array(['チョキ'], dtype='<U3'), count=array([46]))

In [80]:
#過去データに対して結果を予測。predcount回繰り返し、平均値を出力。

df3 = df.copy()
per = []

for i in range(predcount):
    df3['予測結果'] = df3['今回'].apply(lambda x: choose_next_hand(df2, x))
    df3['予測結果'] = df3['予測結果'].shift(1)
    equal_count = (df3['今回'] == df3['予測結果']).sum()
    total_count = len(df3)
    equal_percentage = equal_count / total_count * 100
    per.append(equal_percentage)

np.mean(per)

36.19522643818849

In [81]:
#過去2回のデータで予測
#過去3回のデータの回数をまとめたデータフレームを作成。

jan = ['グー','パー','チョキ']

df4 = pd.DataFrame(columns=['2回前', '1回前', '今回','回数'])

for jan1 in jan:
    for jan2 in jan:
        for jan3 in jan:
            count = ((df1['2回前'] == jan1) & (df1['1回前'] == jan2) & (df1['今回'] == jan3)).sum()
            df4 = df4.append({'2回前': jan1, '1回前': jan2, '今回': jan3, '回数': count}, ignore_index=True)

df4

/var/folders/s_/nhcgq_zn3x7blfst6_jz0mqw0000gn/T/ipykernel_25484/2791511329.py:11: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df4 = df4.append({'2回前': jan1, '1回前': jan2, '今回': jan3, '回数': count}, ignore_index=True)
/var/folders/s_/nhcgq_zn3x7blfst6_jz0mqw0000gn/T/ipykernel_25484/2791511329.py:11: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df4 = df4.append({'2回前': jan1, '1回前': jan2, '今回': jan3, '回数': count}, ignore_index=True)
/var/folders/s_/nhcgq_zn3x7blfst6_jz0mqw0000gn/T/ipykernel_25484/2791511329.py:11: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df4 = df4.append({'2回前': jan1, '1回前': jan2, '今回': jan3, '回数': count}, ignore_index=True)
/var/folders/s_/nhcgq_zn3x7blfst6_jz0mqw0000gn/T/ipykernel_25484/2791511329.py:

,2回前,1回前,今回,回数
0,グー,グー,グー,7
1,グー,グー,パー,46
2,グー,グー,チョキ,56
3,グー,パー,グー,36
4,グー,パー,パー,50
5,グー,パー,チョキ,103
6,グー,チョキ,グー,53
7,グー,チョキ,パー,120
8,グー,チョキ,チョキ,54
9,パー,グー,グー,56


In [82]:
#２回前までの手の出した回数を格納する辞書データを作成。

jan12 = []
dict1 = {}

for jan1 in jan:
    for jan2 in jan:
        jan12.append('{}{}'.format(jan1,jan2))
        
total_count_all = df4['回数'].sum()  
for combo in jan12:
    if combo.startswith('チ'):
        prev2_hand, prev_hand = combo[:3], combo[3:]
    else:
        prev2_hand, prev_hand = combo[:2], combo[2:]
    total_count = df4[(df4['2回前'] == prev2_hand) & (df4['1回前'] == prev_hand)]['回数'].sum()
    
    dict1[combo] = total_count

    
print(dict1)

{'グーグー': 109, 'グーパー': 189, 'グーチョキ': 227, 'パーグー': 192, 'パーパー': 108, 'パーチョキ': 235, 'チョキグー': 224, 'チョキパー': 238, 'チョキチョキ': 110}


In [64]:
#次に出す手の確率を計算する関数

def next_hand_probabilities2(prev_hand, prev2_hand, df, dict1):
    # データフレームから直接値を取得する
    if ('{}{}'.format(prev2_hand, prev_hand)) in dict1:
        prev_total = dict1[prev2_hand + prev_hand]
        
        gu_prob = (df[(df['2回前'] ==  prev2_hand) & (df['1回前'] ==  prev_hand) & (df['今回'] == 'グー')]['回数'].values[0]) / prev_total
        pa_prob = (df[(df['2回前'] ==  prev2_hand) & (df['1回前'] ==  prev_hand) & (df['今回'] == 'パー')]['回数'].values[0]) / prev_total
        cho_prob = (df[(df['2回前'] ==  prev2_hand) & (df['1回前'] ==  prev_hand) & (df['今回'] == 'チョキ')]['回数'].values[0]) / prev_total
        total_prob = gu_prob + pa_prob + cho_prob
        gu_prob /= total_prob
        pa_prob /= total_prob
        cho_prob /= total_prob
        return gu_prob, pa_prob, cho_prob
       
    else:
        # 辞書に対応する組み合わせがない場合はNoneを返すか、適切な処理を行う
        return None
        
#確率に基づき、次の手を予測する
def predict_next_hand2(prev_hand, prev2_hand, df, dict1):
    probabilities = next_hand_probabilities2(prev_hand, prev2_hand, df, dict1)
    next_hand = np.random.choice(['グー', 'パー', 'チョキ'], p=probabilities)
    return next_hand


In [84]:
#今回の予測用のデータフレーム作成

df5 = df.copy()
df5['前回'] = df5['今回'].shift(1)
df5

,今回,前回
0,チョキ,None
1,グー,チョキ
2,グー,グー
3,パー,グー
4,チョキ,パー
...,...,...
1629,チョキ,グー
1630,グー,チョキ
1631,パー,グー
1632,チョキ,パー


In [85]:
#次回の手を予測。predcount回繰り返し最頻値を出力。

prev_hand = df.iloc[-1]['今回']
prev2_hand= df.iloc[-2]['今回']
next_list = []

for i in range(predcount):
    next_hand = predict_next_hand2(prev_hand ,prev2_hand ,df4, dict1)
    next_list.append(next_hand)

stats.mode(next_list)

/var/folders/s_/nhcgq_zn3x7blfst6_jz0mqw0000gn/T/ipykernel_25484/1992631751.py:10: FutureWarning: Unlike other reduction functions (e.g. `skew`, `kurtosis`), the default behavior of `mode` typically preserves the axis it acts along. In SciPy 1.11.0, this behavior will change: the default value of `keepdims` will become False, the `axis` over which the statistic is taken will be eliminated, and the value None will no longer be accepted. Set `keepdims` to True or False to avoid this warning.
  stats.mode(next_list)
/var/folders/s_/nhcgq_zn3x7blfst6_jz0mqw0000gn/T/ipykernel_25484/1992631751.py:10: DeprecationWarning: Support for non-numeric arrays has been deprecated as of SciPy 1.9.0 and will be removed in 1.11.0. `pandas.DataFrame.mode` can be used instead, see https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.mode.html.
  stats.mode(next_list)


ModeResult(mode=array(['グー'], dtype='<U3'), count=array([48]))

In [98]:
#予測結果の表と予測一致度を出力。predcount回繰り返し、平均を出力。

per3 = []

df5['予測結果'] = df5.apply(lambda row: predict_next_hand2(row['今回'],row['前回'] ,df4, dict1), axis=1)
df5['予測結果'] = df5['予測結果'].shift(1)


print(df4)

for i in range(predcount):
    equal_count = (df5['今回'] == df5['予測結果']).sum()
    total_count = len(df5)
    equal_percentage = equal_count / total_count * 100
    per3.append(equal_percentage)

np.mean(per3)

    2回前  1回前   今回   回数
0    グー   グー   グー    7
1    グー   グー   パー   46
2    グー   グー  チョキ   56
3    グー   パー   グー   36
4    グー   パー   パー   50
5    グー   パー  チョキ  103
6    グー  チョキ   グー   53
7    グー  チョキ   パー  120
8    グー  チョキ  チョキ   54
9    パー   グー   グー   56
10   パー   グー   パー   34
11   パー   グー  チョキ  102
12   パー   パー   グー   40
13   パー   パー   パー    6
14   パー   パー  チョキ   62
15   パー  チョキ   グー  123
16   パー  チョキ   パー   62
17   パー  チョキ  チョキ   50
18  チョキ   グー   グー   46
19  チョキ   グー   パー  109
20  チョキ   グー  チョキ   69
21  チョキ   パー   グー  116
22  チョキ   パー   パー   52
23  チョキ   パー  チョキ   70
24  チョキ  チョキ   グー   47
25  チョキ  チョキ   パー   57
26  チョキ  チョキ  チョキ    6


41.003671970624225